# Project 3: Airbnb
**This is the third of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-12-05, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python/PySpark**
- **For each question you may use as many cells for your solution as you like**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**

# Introduction
[Airbnb](http://airbnb.com) is an online marketplace for arranging or offering lodgings. In this project you will use Spark to analyze data obtained from the Airbnb website. The purpose of the analysis is to extract information about trends and patterns from the data.

The project has two parts.

### Part 1: Loading, describing and preparing the data
There's quite a lot of data. Make sure that you can load and correctly parse the data, and that you understand what the dataset contains. You should also prepare the data for the analysis in part two. This means cleaning it and staging it so that subsequent queries are fast.

### Par 2: Analysis
In this part your goal is to learn about trends and usage patterns from the data. You should give solutions to the tasks defined in this notebook, and you should use Spark to do the data processing. You may use other libraries like for instance Pandas and matplotlib for visualisation.

## Guidelines
- Processing data should be done using Spark. Once data has been reduced to aggregate form, you may use collect to extract it into Python for visualisation.
- Your solutions will be evaluated by correctness, code quality and interpretability of the output. This means that you have to write clean and efficient Spark code that will generate sensible execution plans, and that the tables and visualisations that you produce are meaningful and easy to read.
- You may add more cells for your solutions, but you should not modify the notebook otherwise.

### Create Spark session and define imports

In [1]:
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.appName("SparkIntro").getOrCreate()

# Part 1: Loading, describing and preparing the data
The data comes in two files. Start by downloading the files and putting them in your `data/` folder.

- [Listings](https://files.dtu.dk/u/siPzAasj8w2gI_ME/listings.csv?l) (5 GB)
- [Reviews](https://files.dtu.dk/u/k3oaPYp6GjKBeho4/reviews.csv?l) (9.5 GB)

### Load the data
The data has multiline rows (rows that span multiple lines in the file). To correctly parse these you should use the `multiline` option and set the `escape` character to be `"`.

##### Slicing the data

In [2]:
df = spark.read.option("delimiter",",").option("multiline",True).option("header", True).option("escape",'"').option("inferSchema",True).csv('listings.csv')

In [3]:
df.show()

+------+--------------------+--------------+------------+--------------------+--------------------+--------------------+--------------------+-------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+----------+--------------------+--------------+-------+--------------------+---------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+-------------+----------------------+----------------------------+---------+----------------+-------+---------+-----------------+------------+-------+--------+---------+-----------------+-----------------+---------------+------------+---------+--------+----+--------+--------------------+-

In [4]:
df.select("listing_url").show()

+--------------------+
|         listing_url|
+--------------------+
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
|https://www.airbn...|
+--------------------+
only showing top 20 rows



### Describe the data
List the features (schema) and sizes of the datasets.

In [5]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: string (nullable = true)
 |-- last_scraped: string (nullable = true)
 |-- name: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- space: string (nullable = true)
 |-- description: string (nullable = true)
 |-- experiences_offered: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- transit: string (nullable = true)
 |-- access: string (nullable = true)
 |-- interaction: string (nullable = true)
 |-- house_rules: string (nullable = true)
 |-- thumbnail_url: string (nullable = true)
 |-- medium_url: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- xl_picture_url: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: string (nullable = true)
 |-- host_location: string (nullable

In [8]:
df.count()

1330480

### Prepare the data for analysis
You should prepare two dataframes to be used in the analysis part of the project. You should not be concerned with cleaning the data. There's a lot of it, so it will be sufficient to drop rows that have bad values. You may want to go back and refine this step at a later point when doing the analysis.

You may also want to consider if you can stage your data so that subsequent processing is more efficient (this is not strictly necessary for Spark to run, but you may be able to decrease the time you sit around waiting for Spark to finish things)

In [6]:
columns = df.schema.names

In [17]:
df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in columns]).show()

KeyboardInterrupt: 

In [13]:
#df = df.dropna(how='any')

In [14]:
#df.count()

95

# Part 2: Analysis
Use Spark and your favorite tool for data visualization to solve the following tasks.

## The basics
Compute and show a dataframe with the number of listings and neighbourhoods per city.

In [12]:
df_neigh = df.dropna(subset=['neighbourhood_group_cleansed'])

In [15]:
df_neigh.select("id",'city','neighborhood_overview','neighbourhood','neighbourhood_cleansed','neighbourhood_group_cleansed').show()

+--------+--------+---------------------+--------------------+----------------------+----------------------------+
|      id|    city|neighborhood_overview|       neighbourhood|neighbourhood_cleansed|neighbourhood_group_cleansed|
+--------+--------+---------------------+--------------------+----------------------+----------------------------+
|      id|    city| neighborhood_over...|       neighbourhood|  neighbourhood_cle...|        neighbourhood_gro...|
|      id|    city| neighborhood_over...|       neighbourhood|  neighbourhood_cle...|        neighbourhood_gro...|
|31453286|       1|        South Africa"|         Shared room|                     2|                         1.5|
|      id|    city| neighborhood_over...|       neighbourhood|  neighbourhood_cle...|        neighbourhood_gro...|
|  892675|       t| inclusive restaur...|                   4|                     4|        ['email', 'phone'...|
|34244130|    null| Ha 15 min do  Roc...|Pechincha, State ...|                  

In [7]:
columns

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [20]:
group_data = df.groupBy("city")
df_panda = group_data.agg(f.count('id'),f.countDistinct('neighbourhood')).toPandas()

In [40]:
df_panda_nona = df_panda.dropna()

Based on the table above, you should choose a city that you want to continue your analysis for. The city should have mulitple neighbourhoods with listings in them.

Compute and visualize the number of listings of different property types per neighbourhood in your city.

In [53]:
df_panda_nona[df_panda_nona['city'].str.contains('Stockholm')]

,city,count(id),count(DISTINCT neighbourhood)
1182,Stockholm city,1,1
2624,Stockholm,4299,14
5642,Stockholm/Älvsjö,1,1
6052,Stockholm Södermalm,1,1
6519,Hägersten Stockholm,1,1
7270,Stockholm/ Aspudden,1,1
7982,"Stockholm, Bromma",1,1
9837,Stockholm-Hägersten,1,1
9947,Stockholms län,1,1
10169,Stockholm Vällingby,1,1


In [25]:
df_panda.sort_values('count(DISTINCT neighbourhood)')

,city,count(id),count(DISTINCT neighbourhood)
0,Cape town,4,0
12907,Ringwood North,11,0
12908,Pakenham Upper,1,0
12909,Newstead,33,0
12910,Charlotte Cove,2,0
12911,Honeywood,1,0
12912,Moorleah,1,0
12913,Seville Grove,3,0
12914,Mount Nasura,3,0
12915,Azambuja,11,0


In [55]:
prop_neigh_stockholm = df.filter(f.col('city')=='Stockholm').groupby('property_type','neighbourhood').agg(f.count('id')).toPandas()

In [58]:
prop_neigh_stockholm.head()

,property_type,neighbourhood,count(id)


In [56]:
import matplotlib.pyplot as plt
prop_neigh_stockholm.pivot(index='neighbourhood',columns='property_type',\
                      values='count(id)').plot(kind='bar',figsize=(16,6),stacked=True)
plt.show()

TypeError: no numeric data to plot

## Prices
Compute the minimum, maximum and average listing price in your city. 

Compute and visualize the distribution of listing prices in your city.

The value of a listing is its rating divided by its price.

Compute and show a dataframe with the 3 highest valued listings in each neighbourhood.

## Trends
Now we want to analyze the "popularity" of your city. The data does not contain the number of bookings per listing, but we have a large number of reviews, and we will assume that this is a good indicator of activity on listings.

Compute and visualize the popularity (i.e., number of reviews) of your city over time.

Compute and visualize the popularity of neighbourhoods over time. If there are many neighbourhoods in your city, you should select a few interesting ones for comparison.

Compute and visualize the popularity of your city by season. For example, visualize the popularity of your city per month.

## Reviews
In this part you should determine which words used in reviews that are the most positive. 

The individual reviews do not have a rating of the listing, so we will assume that each review gave the average rating to the listing, i.e., the one on the listing.

You should assign a positivity weight to each word seen in reviews and list the words with the highest weight. It is up to you to decide what the weight should be. For example, it can be a function of the rating on the listing on which it occurs, the number of reviews it occurs in, and the number of unique listings for which it was used to review.

Depending on your choice of weight function, you may also want to do some filtering of words. For example, remove words that only occur in a few reviews.